# GitHub Insights

⚠️ This experiments with using [Ibis](https://ibis-project.org/) for analysis.

## Imports

In [2]:
%pip install ibis-framework[duckdb]

import ibis
from ibis import _

ibis.options.interactive = True

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Load data

In [3]:
con = ibis.connect("duckdb://")

start = ibis.date("2024-05-12")
end = ibis.date("2024-08-10")

top_issues = con.read_json("top_issues/flutter/flutter/2024-08-*.jsonl")

## Trending issues

In [4]:
top_issues \
.filter(_.date.between(start, end)) \
.group_by([_.id]) \
.aggregate(
  name = _.title.argmax(_.date),
  reactions_start = _.reactions.argmin(_.date),
  reactions_end = _.reactions.argmax(_.date),
) \
.mutate(reactions_change = _.reactions_end - _.reactions_start) \
.select(_.id, _.name, total_reactions = _.reactions_end, reactions_change = _.reactions_change) \
.order_by(_.reactions_change.desc())

top_issues

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ date       ┃ repository      ┃ id    ┃ title                                                        ┃ state  ┃ comments ┃ participants ┃ reactions ┃ createdAt                ┃ labels ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ date       │ string          │ int64 │ string                                                       │ string │ int64    │ int64        │ int64     │ string                   │ array… │
├────────────┼─────────────────┼───────┼──────────────────────────────────────────────────────────────┼────────┼──────────┼──────────────┼───────────┼──────────────────────────┼────────┤
│ 2024-08-01 │ flutter/flutter │ 14330 │ Code Push / Hot Update / out of band updates                 │ OPEN   │      285 │          164 │      1987 │ 2018-01-29T20:40:17.000Z │ NULL   │
│ 2024-08-01 │ flutter/flutter │ 53041 │ Enable "hot reload" (not just "hot restart") for Flutter Web │ OPEN   │      151 │           73 │      1105 │ 2020-03-22T09:27:31.000Z │ NULL   │
│ 2024-08-01 │ flutter/flutter │ 14050 │ Let flutter be installable via homebrew                      │ OPEN   │       83 │           53 │      1005 │ 2018-01-11T19:00:27.000Z │ NULL   │
│ 2024-08-01 │ flutter/flutter │ 46789 │ Improve the indexability (SEO) of Flutter apps on the web    │ OPEN   │      196 │           85 │       784 │ 2019-12-11T17:11:17.000Z │ NULL   │
│ 2024-08-01 │ flutter/flutter │ 41722 │ Implement PlatformView support on macOS                      │ OPEN   │       64 │           35 │       648 │ 2019-10-01T17:57:26.000Z │ NULL   │
│ 2024-08-01 │ flutter/flutter │ 47600 │ Server-side rendering for Flutter web                        │ OPEN   │       59 │           42 │       630 │ 2019-12-22T02:00:49.000Z │ NULL   │
│ 2024-08-01 │ flutter/flutter │ 30701 │ Support multiple windows for desktop shells                  │ OPEN   │      132 │           79 │       619 │ 2019-04-08T16:33:22.000Z │ NULL   │
│ 2024-08-01 │ flutter/flutter │ 26801 │ Apple CarPlay / Android Auto support?                        │ OPEN   │       53 │           43 │       505 │ 2019-01-19T09:29:46.000Z │ NULL   │
│ 2024-08-01 │ flutter/flutter │ 94340 │ [Feature Request] Support for Jetbrains Fleet\n              │ OPEN   │       19 │            6 │       438 │ 2021-11-29T13:02:17.000Z │ NULL   │
│ 2024-08-01 │ flutter/flutter │ 24213 │ [google_maps_flutter] Support Widgets as markers             │ OPEN   │       54 │           42 │       433 │ 2018-11-10T16:08:29.000Z │ NULL   │
│ …          │ …               │     … │ …                                                            │ …      │        … │            … │         … │ …                        │ …      │
└────────────┴─────────────────┴───────┴──────────────────────────────────────────────────────────────┴────────┴──────────┴──────────────┴───────────┴──────────────────────────┴────────┘

## Trending issues by team

### Framework

`team-framework` issues that received the most reactions

In [5]:
label = "team-framework"

issues_with_label = \
top_issues \
.mutate(id = _.repository.concat("#", _.id.cast("string"))) \
.group_by([_.id]) \
.aggregate(labels = _.labels.argmax(_.date)) \
.filter(_.labels.contains(label)) \
.select(_.id)

top_issues \
.filter(_.date.between(start, end)) \
.mutate(id = _.repository.concat("#", _.id.cast("string"))) \
.filter(_.id.isin(issues_with_label.id)) \
.group_by([_.id]) \
.aggregate(
  name = _.title.argmax(_.date),
  reactions_start = _.reactions.argmin(_.date),
  reactions_end = _.reactions.argmax(_.date),
) \
.mutate(recent_reactions = _.reactions_end - _.reactions_start) \
.select(_.id, _.name, total_reactions = _.reactions_end, recent_reactions = _.recent_reactions) \
.order_by(_.recent_reactions.desc())

┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ id                     ┃ name                                                                   ┃ total_reactions ┃ recent_reactions ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ string                 │ string                                                                 │ int64           │ int64            │
├────────────────────────┼────────────────────────────────────────────────────────────────────────┼─────────────────┼──────────────────┤
│ flutter/flutter#59284  │ Text widget incorrectly wraps Korean text                              │              34 │                9 │
│ flutter/flutter#151065 │ Proposal: Framework needs to be aware of physical pixels               │             151 │                6 │
│ flutter/flutter#127855 │ Implement wide gamut color support in the Framework                    │              68 │                6 │
│ flutter/flutter#30701  │ Support multiple windows for desktop shells                            │             624 │                5 │
│ flutter/flutter#102003 │ Rewrite the NestedScrollView                                           │              86 │                5 │
│ flutter/flutter#65504  │ Ctrl+F support, finding text on a page (even when scrolled off screen) │             145 │                3 │
│ flutter/flutter#54200  │ allow Hero widgets to have transitions within the same screen          │             224 │                2 │
│ flutter/flutter#97544  │ SliverWrap widget                                                      │              48 │                2 │
│ flutter/flutter#40675  │ Add `bool validateOnLostFocus` to the `FormField` and `Form` class     │              23 │                2 │
│ flutter/flutter#32045  │ Flutter clipboard should support images                                │              91 │                2 │
│ …                      │ …                                                                      │               … │                … │
└────────────────────────┴────────────────────────────────────────────────────────────────────────┴─────────────────┴──────────────────┘

`